In [1]:
%run NB_Credentials

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 3, Finished, Available, Finished)

In [2]:
import requests
import pandas as pd
import time
import re

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 4, Finished, Available, Finished)

In [3]:
# Iniciar tiempo de ejecución
start_time = time.time()

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 5, Finished, Available, Finished)

In [4]:
# Parámetros del Data Lake

TABLA_ORIGEN = "bronze_fact_post"
TABLA_DESTINO = "bronze_dim_tags"

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 6, Finished, Available, Finished)

In [5]:
# Expresión regular para extraer hashtags
HASHTAG_PATTERN = re.compile(r"#\w+")

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 7, Finished, Available, Finished)

In [6]:
# Leer la tabla fact_post para obtener los posts
df_fact_post = spark.read.format("delta").load(f"{BRONZE_PATH}/{TABLA_ORIGEN}").select("author", "post", "date")
print(f"🔄 Total de posts a procesar: {df_fact_post.count()}")

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 8, Finished, Available, Finished)

🔄 Total de posts a procesar: 438


In [7]:
# Convertir a Pandas para procesamiento
df_pandas = df_fact_post.toPandas()

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 9, Finished, Available, Finished)

In [8]:
# Lista donde almacenaremos los hashtags extraídos
tags_data = []

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 10, Finished, Available, Finished)

In [9]:
# Iterar sobre los posts para extraer hashtags
for _, row in df_pandas.iterrows():
    hashtags = list(set(HASHTAG_PATTERN.findall(row["post"])))  # Extraer hashtags sin duplicados
    
    if hashtags:
        first_hashtag = hashtags[0]  # Primer hashtag
        related_hashtags = ", ".join(hashtags[1:]) if len(hashtags) > 1 else ""  # Resto de hashtags
        
        tags_data.append({
            "date": row["date"],
            "author": row["author"],
            "hashtag": first_hashtag,
            "related_hashtags": related_hashtags
        })

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 11, Finished, Available, Finished)

In [10]:
# Convertir a DataFrame de Spark
df_tags = spark.createDataFrame(tags_data)

# Mostrar resultado 
# display(df_tags)

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 12, Finished, Available, Finished)

In [11]:
# Guardar en Lakehouse en formato Delta
df_tags.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{BRONZE_PATH}/{TABLA_DESTINO}")

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 13, Finished, Available, Finished)

In [12]:
# Calcular tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"✅ Proceso completado en {execution_time:.2f} segundos.")

StatementMeta(, a0ed6bf0-45d7-487f-8221-e1666e93fc00, 14, Finished, Available, Finished)

✅ Proceso completado en 24.06 segundos.
